# Section 3

## Review of Language Modeling and Neural Networks

- Language model using bigrams - `p(current_word | previous_word)`. This is a Markov model, a probabilistic model in which predictions are made based only on current state.
- Model previous language in a neural version. Logistic regression.
- Neural network of the previous model.
- Improve efficiency of the above with an implementation trick.

### Bigrams and language models

- Language model: a model of the probability of a sequence of words
  - given a sentence `s`, the language model gives `p(s)`
  - creating the model typically involve making assumptions about the structure of a specific language
- **Note:** A model is never 100% correct, it has assumptions, which may be:
  - Correct most of the times, incorrect sometimes
  - Incorrect most of the times, but still powerfull. 
- Example: a map. "The map is not the territory"

- Bigram: two consecutive words in a sentence
- Trigrams: three consecutive words in a sentence
- N-grams: sequence of `n` consecutive words
- Bigram model: `p(w_t | w_t-1)`
    - given a set of documents
    - build the model by counting: # appearances `w_t-1 w_t` / # appearances `w_t-1`
- Set of documents: it a file of files containing sentences. Training corpus. 

- Using Bigram models we'll build a language model: 
    - Bayes rule: `p(A -> B -> C) = p(C|A -> B) * p(A -> B) = p(C|A -> B) * p(B|A) * p(A)`
    - Chain rule of probability
    - `p(B|A)` is a Bigram model
    - `p(C|A -> B) = count(A -> B -> C) / count(A -> B)`
    - `p(A) = count(A) / corpus length`
    - For long sentences this becomes problematic
    
- Long sentences, say `"A B C D E F"`, may lead to `p(G| A B C D E) = 0` which may not be true. 
    - Add-one smoothing: `p_smooth(B|A) = (count(A -> B) + 1) / (count(A) + V)`
    - V: vocabulary size = number of distinct words
    - Adding V to the denominator makes the probability valid, i.e. summing all values gives 1.
    - Also this smoothing allows for not having a probability of 0 for any pair A and B. 
    
- Markov Assumption
    - What I see now depends *only* on what I saw in the previous step.
    - `p(w_t | w_t-1, w_t-2, ..., w_1) = p(w_t | w_t-1)`
    - Second, third, ... order Markov. 
    - `p(A B C D E) = p(E | D) * p(D | C) * p(C | B) * p(B | A) * p(A)`
    - All elements are bigrams!

- Long sentences are rare, but small sentences, like bigrams, may be really common. 

Load the Brown corpus using NLTK

In [1]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [2]:
from nltk.corpus import brown
len(brown.sents())

57340

In [4]:
sentences = brown.sents()
print('\n\n'.join(' '.join(s) for s in sentences[:5]))

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .

The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .

The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. .

`` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .

The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '' .


Create a bigram language model with add-one smoothing. 
- Use lower case preprocessing.

Hints: 
- Use log probabilities to avoid underflow to 0
- Normalize each sentence, by dividing by their length $T$

$$
\frac{1}{T} log p(w_1, ..., w_T) = \frac{1}{T} \left[ log p(w_1) + \sum^{T}_{t=2} log p (w_t | w_t-1) \right]
$$

In [166]:
import math

In [191]:
class MarkovBigramLanguageModel:
    def __init__(self, language, smooth = True):
        try:
            self.stopwords = stopwords.words(language)
        except:
            self.stopwords = []
        self.smooth = smooth
        self.wordCount = {}
        self.wordProb = {}
        self.bigramCount = {}
        self.bigramProb = {}
    
    def train(self, sentences):
        for sentence in sentences:
            normSentence = self._preprocessSentence(sentence)
            if len(normSentence) == 0: 
                continue
            self._addWordCount(normSentence[0])
            for i in range(1, len(normSentence)):
                prevWord = normSentence[i-1]
                curWord = normSentence[i]
                self._addWordCount(curWord)
                self._addBigramCount(prevWord, curWord)
        self._calcProbabilityMatrices()
    
    def probability(self, sentence):
        normSentence = self._preprocessSentence(sentence)
        if len(normSentence) == 0:
            return 0.0
        prob = self.wordProb[normSentence[0]]
        for i in range(1, len(normSentence)):
            prevWord = normSentence[i-1]
            curWord = normSentence[i]
            if prevWord in self.bigramProb and curWord in self.bigramProb[prevWord]:
                prob += self.bigramProb[prevWord][curWord]
        return prob / len(normSentence)
    
    def _calcProbabilityMatrices(self):
        vocabSize = float(len(self.wordCount))
        for w in self.wordCount:
            self.wordProb[w] = self.wordCount[w] / vocabSize
        for w0 in self.bigramCount:
            self.bigramProb[w0] = {}
            for w1 in self.bigramCount[w0]:
                if (self.smooth):
                    bp = (self.bigramCount[w0][w1] + 1.0)/ (self.wordCount[w0] + vocabSize)
                else:
                    bp = self.bigramCount[w0][w1] / self.wordCount[w0]
                self.bigramProb[w0][w1] = bp
        
    def _addWordCount(self, inputWord):
        if inputWord not in self.wordCount:
            self.wordCount[inputWord] = 0.0
        self.wordCount[inputWord] += 1.0

    def _addBigramCount(self, prevWord, curWord):
        if prevWord not in self.bigramCount:
            self.bigramCount[prevWord] = {}
        if curWord not in self.bigramCount[prevWord]:
            self.bigramCount[prevWord][curWord] = 0.0
        self.bigramCount[prevWord][curWord] += 1.0
        
    def _preprocessSentence(self, sentence):
        preprocessedWords = [self._preprocessWord(w) for w in sentence]
        normSentence = []
        for w in preprocessedWords:
            if w not in self.stopwords:
                normSentence.append(w)
        return normSentence

    def _preprocessWord(self, word):
        return word.lower()

In [129]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Test your model
- Compare the probability of a real sentence from the corpus vs. a fake sentence (randomly generated words)
- Compare a fake sentence vs. a custom valid sentence written by me.

In [187]:
import random

def generateTestSentences():
    words = list(brown.words())
    sentences = list(brown.sents())
    random.shuffle(words)
    corpusSentence = random.choice(sentences)
    fakeSentence = words[:10]
    realSentence = "I ride my bike on the mountain".split(' ')
    return {
        'corpus': corpusSentence,
        'fake': fakeSentence,
        'real': realSentence,
    }

def testModel(model, testSentences):
    report = {}
    for t in testSentences: 
        sentence = testSentences[t]
        prob = model.probability(sentence)
        report[t] = (prob, ' '.join(sentence))
    return report

In [188]:
trainSentences = brown.sents()
testSentences = generateTestSentences()

In [184]:
model1 = MarkovBigramLanguageModel(language='english', smooth=True)
model1.train(trainSentences)

In [185]:
model2 = MarkovBigramLanguageModel(language='english', smooth=False)
model2.train(trainSentences)

In [192]:
model3 = MarkovBigramLanguageModel(language='', smooth=True)
model3.train(trainSentences)

In [193]:
model4 = MarkovBigramLanguageModel(language='', smooth=False)
model4.train(trainSentences)

In [199]:
reportStopWordsSmooth = testModel(model1, testSentences)
reportStopWordsNoSmooth = testModel(model2, testSentences)
reportAllWordsSmooth = testModel(model3, testSentences)
reportAllWordsNoSmooth = testModel(model4, testSentences)

In [200]:
reportStopWordsSmooth

{'corpus': (0.002133892069761611,
  u'They are still considered to be for use in restricted waters , however , and targets must come within a few yards of them .'),
 'fake': (0.0002711037433867193,
  u'witnesses board despise Administration both was Give his , .'),
 'real': (0.00032893632732521066, 'I ride my bike on the mountain')}

In [201]:
reportStopWordsNoSmooth

{'corpus': (0.10022049539158925,
  u'They are still considered to be for use in restricted waters , however , and targets must come within a few yards of them .'),
 'fake': (0.002976413139989482,
  u'witnesses board despise Administration both was Give his , .'),
 'real': (0.00032893632732521066, 'I ride my bike on the mountain')}

In [202]:
reportAllWordsSmooth

{'corpus': (0.007645759363363574,
  u'They are still considered to be for use in restricted waters , however , and targets must come within a few yards of them .'),
 'fake': (7.529510808173558e-05,
  u'witnesses board despise Administration both was Give his , .'),
 'real': (0.021063662621245625, 'I ride my bike on the mountain')}

In [203]:
reportAllWordsNoSmooth

{'corpus': (0.08072599697821253,
  u'They are still considered to be for use in restricted waters , however , and targets must come within a few yards of them .'),
 'fake': (0.0017091119743144491,
  u'witnesses board despise Administration both was Give his , .'),
 'real': (0.06712548302474791, 'I ride my bike on the mountain')}